In [1]:
import numpy as np        # check NumPy
import tinyarray          # check tinyarray|

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import kwant              # check Kwant core
from scipy.linalg import eigh
from scipy.constants import hbar, e, k as k_B

In [3]:
from tqdm.notebook import tqdm
import time

In [4]:
# Run this in a Jupyter cell
import sys
print(sys.executable)  # Should point to your conda environment's Python

/Users/jingyili/miniforge/envs/kwant-intel/bin/python


In [5]:
# Constants
a = 1.0           # Lattice constant (nm)
t = 1.0           # Hopping (eV)
mu = 0.0          # Chemical potential (eV)
alpha = 0.5       # Rashba SOC (eV·nm)
h0 = 0.5          # Exchange field (eV)
W = 50e-9         # Junction width (m)
hbar = 6.582119569e-16  # eV·s (reduced Planck constant)
e = 1.602176634e-19     # C (elementary charge)
phi0 = hbar/(2*e) # Flux quantum (Wb)
k_B = 8.617333262e-5    # Boltzmann constant (eV/K)
Lx, Ly = 100, 20  # System dimensions

In [6]:
def make_system():
     # Define Pauli matrices HERE
    sigma_0 = tinyarray.array([[1, 0], [0, 1]])
    sigma_x = tinyarray.array([[0, 1], [1, 0]])
    sigma_y = tinyarray.array([[0, -1j], [1j, 0]])
    sigma_z = tinyarray.array([[1, 0], [0, -1]])
    tau_0 = sigma_0
    tau_x = tinyarray.array([[0, 1], [1, 0]])
    tau_z = tinyarray.array([[1, 0], [0, -1]])
    
    lat = kwant.lattice.square(a, norbs=4)  # Add norbs=4
    sys = kwant.Builder()

    def onsite(site, Delta_array, phase, B):
        x, y = site.pos
        x_idx, y_idx = int(round(x/a)), int(round(y/a))
        if x < 40*a or x >= 60*a:
            # S regions: fixed Δ with phase
            Delta_val = 0.2 * np.exp(1j * phase * (-1 if x < 40*a else 1))
            h = 0.0
        else:
            # F region: use Delta_array
            Delta_val = Delta_array[x_idx, y_idx]
            h = h0 * np.tanh((x - 50*a)/(10*a))  # Domain wall
        onsite_matrix = (
            (4*t - mu) * np.kron(sigma_0, tau_z) +
            h * np.kron(sigma_z, tau_z) +
            np.real(Delta_val) * np.kron(sigma_0, tau_x) +
            np.imag(Delta_val) * np.kron(sigma_0, np.dot(tau_x, sigma_y))  # Use np.dot for matrix multiplication
        )
        return onsite_matrix

    def hopping_x(site1, site2, B):
        y = site1.pos[1]
        flux_phase = 2*np.pi * B * y * a**2 / phi0
        return (
            -t * np.kron(sigma_0, tau_z) -
            1j*alpha * np.kron(sigma_y, tau_z)
        ) * np.exp(1j*flux_phase)

    def hopping_y(site1, site2):
        return (
            -t * np.kron(sigma_0, tau_z) +
            1j*alpha * np.kron(sigma_x, tau_z)
        )

    # Build system
    sys[lat.shape((lambda pos: 0 <= pos[0] < Lx*a and 0 <= pos[1] < Ly*a), (0, 0))] = onsite
    sys[lat.neighbors()] = hopping_x
    sys[kwant.HoppingKind((0, 1), lat)] = hopping_y

    return sys.finalized()

In [7]:
# start self consistently updateing Delta, the order parameter 
def self_consistent_Delta(sys, V=1.0, T=0.01, max_iter=100, tol=1e-4, phase=0, B=0):
   # Initialization code
    print("\nStarting self-consistent calculation...")
    print(f"Target tolerance: {tol:.1e}, Max iterations: {max_iter}")
    start_time = time.time()
    
    # Initialize Δ (this should be OUTSIDE the iteration loop)
    Delta = np.zeros((Lx, Ly), dtype=complex)
    # Set up S regions (do this once before iterations)
    for x in range(Lx):
        x_pos = x * a
        for y in range(Ly):
            if x_pos < 40*a or x_pos >= 60*a:
                phase_factor = -1 if x_pos < 40*a else 1
                Delta[x, y] = 0.2 * np.exp(1j * phase * phase_factor)

    
    with tqdm(total=max_iter, desc="SCF Cycle", unit="iter", 
             bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]") as pbar:
        for iter in range(max_iter):
            # Main calculation code here
            params = {'Delta_array': Delta, 'phase': phase, 'B': B}
            H = sys.hamiltonian_submatrix(params=params)
            eps, psi = eigh(H)
            
            Delta_new = Delta.copy()  # Start with current Δ

            # Reset F region to accumulate new Δ
            mask = (np.arange(Lx)*a >= 40*a) & (np.arange(Lx)*a < 60*a)
            Delta_new[mask, :] = 0.0

            for n in range(len(eps)):
                vec = psi[:, n]
                weight = np.tanh(0.5 * abs(eps[n]) / (k_B * T)) if T > 0 else 1.0
            
                for x in range(Lx):
                    if not mask[x]:
                        continue  # Skip S regions
                    for y in range(Ly):
                        idx = y + Ly * x
                        u_up = vec[4*idx]
                        u_dn = vec[4*idx + 1]
                        v_up = vec[4*idx + 2]
                        v_dn = vec[4*idx + 3]
                        contrib = V * (u_up * v_dn.conj() + u_dn * v_up.conj()) * weight
                        Delta_new[x, y] += contrib

            # Apply particle-hole symmetry and mixing
            Delta_new[mask] *= 0.5 / Ly  # Normalize by number of sites (adjust based on your model)
            max_diff = np.max(np.abs(Delta_new[mask] - Delta[mask]))
            Delta[mask] = Delta_new[mask]  # Update only F region

            # Update progress bar
            pbar.set_postfix({'ΔΔ': f"{max_diff:.2e}",
                              '|Δ|': f"{np.mean(np.abs(Delta[mask])):.3e}"})
            pbar.update(1)
        
            if max_diff < tol:
                print(f"Converged after {iter+1} iterations")
                break
    # ... [final reporting] ...
    print(f"\nTotal time: {time.time()-start_time:.1f}s")
    return Delta

In [8]:
sys = make_system()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
Delta = self_consistent_Delta(sys, V=1.0, T=0.01, phase=0, B=0)


Starting self-consistent calculation...
Target tolerance: 1.0e-04, Max iterations: 100


SCF Cycle:   0%|          | 0/100 [00:00<?]

In [8]:
def compute_critical_current(B_values):
    I_c = []
    for B in B_values:
        # Self-consistent solution
        sys = make_system(B=B)
        Delta = self_consistent_Delta(sys)
        
        # Current-phase relation
        phases = np.linspace(0, 2*np.pi, 50)
        currents = []
        for phi in phases:
            sys_phi = make_system(B=B, phase=phi)
            H = sys_phi.hamiltonian_submatrix()
            eps = eigh(H, eigvals_only=True)
            E_gs = np.sum(eps[eps < 0])
            currents.append(2*e/hbar * np.gradient(E_gs, phi))
        
        I_c.append(np.max(np.abs(currents)))
    
    return np.array(I_c)

In [9]:
# Plot Δ(x, y)
import matplotlib.pyplot as plt
plt.imshow(np.abs(Delta), cmap='viridis')
plt.colorbar(label='|Δ(x, y)|')

# Plot h(x)
plt.plot(np.arange(Lx)*a, h)
plt.xlabel("x (nm)")
plt.ylabel("h(x) (eV)")

NameError: name 'Delta' is not defined

In [11]:
# Generate Fraunhofer pattern
B_values = np.linspace(-2, 2, 50)  # In units of Φ/Φ₀
I_c = compute_critical_current(B_values)

# Plot
import matplotlib.pyplot as plt
plt.plot(B_values, I_c)
plt.xlabel("Magnetic Field (Φ/Φ₀)")
plt.ylabel("Critical Current I_c")
plt.title("Asymmetric Fraunhofer Pattern")
plt.show()

/Users/jingyili/miniforge/envs/kwant-intel/lib/python3.10/site-packages/kwant/lattice.py:746: KwantDeprecationWarning: Not specfying norbs is deprecated. Always specify norbs when creating site families.
  return Monatomic(((a, 0), (0, a)), name=name, norbs=norbs)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


UserCodeError: Error occurred in user-supplied value function "onsite".
See the upper part of the above backtrace for more information.